# Basic finite volume solver for scalar hyperbolic conservation laws

## Prerequisites (libraries and helper functions)
We start loading several libraries:

In [ ]:
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
ngsglobals.msg_level = 1
from ngsolve.meshes import *
from draw import *


We may need function for absolute value and maxima computations on `CoefficientFunctions` which we compose with the `IfPos` function:

In [ ]:
def Abs(u):
    return IfPos(u,u,-u)

def Max(u,v):
    return IfPos(u-v,u,v)

## A simple finite volume solver

We can now define a very simple finite volume solver based on:
 * a flux function $F$,
 * a numerical flux function $\hat{f}_n$,
 * initial values $u_0$,
 * boundary data (if needed),
 * a mesh and
 * a time step for an explicit Euler discretization.
 
Let $u_T$ be the constant corresponding to one element. Then the scheme reads as

$$
u_T^{n+1} = u_{T}^n - \frac{\Delta t}{|T|} \sum_{K \in \partial T} 
\hat{f}_{K,n}(u_T,u_{T'},n_K) |K|
$$
    

This, together with some plotting at $t = 0,0.1,..,0.5$ is done in the following function:

In [ ]:
def Solve(F, fhatn, u0, ubnd, mesh, dt):
    V = L2(mesh,order=0)
    gfu=GridFunction(V)
    u,v = V.TnT()
    # definition of the "bilinear" form
    # Note: It -- despite its name -- does not need to be linear in the first argument!
    a = BilinearForm (V, nonassemble=True)
    #a += -F(u)*grad(v) * dx #<- only for DG
    a += fhatn(F,u,u.Other(ubnd),specialcf.normal(mesh.dim)) * v * dx(element_boundary=True)
    t=0

    gfu.Set(u0)
    intu0 = Integrate(gfu,mesh,order=0)
    Draw(gfu,mesh,"u")
    
    Ts = [0.1,0.4,1.6]
    for T in Ts:
        while t < T-dt/2:
            gfu.vec.data -= dt * V.InvM() @ a.mat * gfu.vec
            t += dt
            Redraw()
        print("t = ", t, "conservation error: ", abs(Integrate(gfu,mesh,order=0)-intu0))
        Draw(gfu,mesh,"u")
    return gfu

## Some helper functions:
* Computing the Jacobian of a scalar flux
* Computing the maximum char. speed between two states $u_1, u_2$

(Note that in general the maximum char. speed could also be assumed in the interior $(\min(u_1,u_2),\max(u_1,u_2))$. In that case the function `AbsFmax` should be overwritten manually)

In [ ]:
def Jacobian(F,u):
    dummy = Parameter(1)
    return F(dummy).Derive(dummy,u)
def AbsFmax(F,u1,u2):
    absdfus = [Norm(Jacobian(F,u)) for u in [u1,u2]]
    return Max(absdfus[0],absdfus[1])

## The mesh
For the mesh we take a N(xN) mesh. The time step is manually adjusted to provide stability in the sense of a CFL condition:

In [ ]:
N=50
#mesh = MakeStructured2DMesh(nx=N,ny=N,quads=True,periodic_x=True,periodic_y=True)
mesh = Make1DMesh(n=N,periodic=True)
dt=0.5/N

The domain will be $\Omega = [0,1]^d$ with periodic boundary conditions everywhere. For the purpose of having the feature available for later we also give the code to provide boundary conditions here:

(possible) boundary conditions:

In [ ]:
ubnd_dir = {"bottom" : 0, "right" : 0, "top" : 0, "left" : 0}

In [ ]:
ubnd = CoefficientFunction([ubnd_dir[key] for key in mesh.GetBoundaries()])

## The problem
As the model problem here we consider (a trivial extension of) the Burgers equation (to 2D):
$$
F(u(x)) = \frac12 u^2 \text{ in 1D } \quad \text{ or } \quad
F(u(x,y,t)) = (\frac12 u^2, \frac12 u^2)^T \text{ in 2D }
$$

In [ ]:
def F(u):
    return CoefficientFunction(tuple([0.5*u*u for i in range(mesh.dim)]))

As initial values we prescribe $u_0(x(,y)) = \sin(2\pi x)$:

In [ ]:
if mesh.dim == 2:
    u0 = sin(2*pi*x)*cos(2*pi*y)
else:
    u0 = IfPos(x-0.5,1,0)

## numerical examples

### A discretization with a central flux

In [ ]:
def fhatn_central(F,u1,u2,n):
    return 0.5*F(u1)*n+0.5*F(u2)*n # central flux

### A discretization with a Lax-Friedrichs flux

We define a flux function `fhatn_LF(u1,u2,n)` as a Lax-Friedrichs flux:
$$
\hat{f}_{K,n}(u_1,u_2,n) = \frac{F(u_1)+F(u_2)}{2} \cdot n+ |F_{\max}| \frac{u_1-u_2}{2}
$$
where $|F_{\max}| = \max_{v \in \{u_1,u_2\}} \Vert \frac{\partial F}{\partial u}(v) \Vert_2$:

In [ ]:
def fhatn_LF(F,u1,u2,n):
    return fhatn_central(F,u1,u2,n) + AbsFmax(F,u1,u2) *0.5*(u1-u2)
gfu = Solve(F,fhatn_LF, u0, ubnd, mesh, dt)

### Draw direction of characteristics:

In [ ]:
Draw(Jacobian(F,gfu),mesh,"direction of characteristics")